In [1]:
# notebooks/06_Analisis_de_Sentimientos.ipynb
import os
import logging
import sys
import pandas as pd
# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from utils.file_utils import read_excel_file, save_data_to_excel
from config import XLSX_DIRECTORY

def analyze_sentiment_vader(sentence):
    sentiment_analyzer = SentimentIntensityAnalyzer()
    sentiment_score = sentiment_analyzer.polarity_scores(sentence)
    return sentiment_score['compound']  # Devuelve la puntuación compuesta

def main():
    # Cargar los datos de las oraciones desde el archivo Excel
    logging.info("Cargando datos de oraciones desde Excel para análisis de sentimientos...")
    sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))

    # Preparar DataFrame para los resultados del análisis de sentimientos
    sentiment_df = pd.DataFrame(columns=["id_sentiment", "id_sentence",  "sentiment_score"])

    # Realizar análisis de sentimientos
    logging.info("Realizando análisis de sentimientos...")
    for index, row in sentence_data.iterrows():
        sentence = str(row["sentence_clean"])  # Convertir a cadena de texto
        sentiment_score = analyze_sentiment_vader(sentence)
        sentiment_df.loc[index] = [index + 1, row["id_sentence"], sentiment_score]  # index + 1 para iniciar en 1

    # Guardar los resultados del análisis de sentimientos en un nuevo archivo Excel
    logging.info("Guardando resultados del análisis de sentimientos en Excel...")
    save_data_to_excel(sentiment_df, XLSX_DIRECTORY, "sentiment.xlsx")

if __name__ == "__main__":
    main()

In [ ]:
#pip install vaderSentiment
